In [1]:
import numpy as np
import scipy.linalg as linear

#QR Methods
def simpleQR(A, tol, maxIter=1e3, symmetric=False):
    i = 0
    if not symmetric:
        while i < maxIter and linear.norm(A.diagonal(-1))>tol:
            [Q,R] = linear.qr(A)
            A = R.dot(Q)
            i += 1
        return A,i
    else:
        V=np.identity(len(A))
        while i < maxIter and linear.norm(A.diagonal(-1))>tol:
            [Q,R] = linear.qr(A)
            A = R.dot(Q)
            V = V.dot(Q)
            i += 1
        return A,V,i

def shiftQRStep(A, tol, symmetric=False):
    i = 0
    if not symmetric:
        while abs(A[-1,-2])>tol:
            shift = A[-1,-1]
            [Q,R] = linear.qr(A-shift*np.identity(len(A)))
            A = R.dot(Q)+shift*np.identity(len(A))
            i += 1
        return A,i
    else:
        #Acumular eigenvectores
        while abs(A[-1,-2])>tol:
            delta = 0.5*(A[-2,-2]-A[-1,-1])
            shift = A[-1,-1] - np.sign(delta)*((A[-2,-1]**2)/(abs(delta)+np.sqrt(delta**2+A[-2,-1]**2)))
            [Q,R] = linear.qr(A-shift*np.identity(len(A)))
            A = R.dot(Q)+shift*np.identity(len(A))
            i += 1
        return A,i

def shiftQR(A, tol, maxIter=1e3, symmetric=False):
    i = 0
    A = linear.hessenberg(A)
    eigenvalues = []
    if not symmetric:
        while i<maxIter and len(A)>1:
            [A,j] = shiftQRStep(A,tol)
            eigenvalues.append(A[-1,-1])
            A = A[:-1,:-1]
            i += j
        eigenvalues.append(A[0,0])
        return eigenvalues,i
    else:
        while i<maxIter and len(A)>1:
            [A,j] = shiftQRStep(A,tol, True)
            eigenvalues.append(A[-1,-1])
            A = A[:-1,:-1]
            i += j
        eigenvalues.append(A[0,0])
        return eigenvalues,i

In [2]:
A=np.array([[0,1,2],[1,0,1],[2,1,0]])
print("A")
print(A)

A
[[0 1 2]
 [1 0 1]
 [2 1 0]]


In [3]:
linear.eig(A)

(array([ 2.73205081+0.j, -2.00000000+0.j, -0.73205081+0.j]),
 array([[ -6.27963030e-01,  -7.07106781e-01,   3.25057584e-01],
        [ -4.59700843e-01,   1.64352863e-16,  -8.88073834e-01],
        [ -6.27963030e-01,   7.07106781e-01,   3.25057584e-01]]))

In [4]:
simpleQR(A,1e-10)

(array([[  2.73205081e+00,   7.76052179e-11,   4.52172420e-16],
        [  7.76065272e-11,  -2.00000000e+00,   3.46344160e-16],
        [  1.71922142e-45,  -3.32295778e-35,  -7.32050808e-01]]), 80)

In [5]:
shiftQRStep(A,1e-10)

(array([[  2.72321536e+00,   2.04283468e-01,   1.93389107e-14],
        [  2.04283468e-01,  -1.99116455e+00,  -1.44199598e-13],
        [  1.97668268e-14,  -1.44287698e-13,  -7.32050808e-01]]), 4)

In [6]:
shiftQR(A,1e-10)

([-0.73205080756887753, -1.9999999999999998, 2.7320508075688763], 5)

In [7]:
B=np.array([[0,1,2],[1,0,1],[2,1,0]])
print("B")
print(B)

B
[[0 1 2]
 [1 0 1]
 [2 1 0]]


In [8]:
simpleQR(B,1e-10,symmetric=True)

(array([[  2.73205081e+00,   7.76052179e-11,   4.52172420e-16],
        [  7.76065272e-11,  -2.00000000e+00,   3.46344160e-16],
        [  1.71922142e-45,  -3.32295778e-35,  -7.32050808e-01]]),
 array([[  6.27963030e-01,  -7.07106781e-01,   3.25057584e-01],
        [  4.59700843e-01,   7.53884818e-12,  -8.88073834e-01],
        [  6.27963030e-01,   7.07106781e-01,   3.25057584e-01]]),
 80)

In [9]:
shiftQRStep(B,1e-10, symmetric=True)

(array([[  2.72539799e+00,   1.77305411e-01,  -1.24562699e-16],
        [  1.77305411e-01,  -1.99334718e+00,   7.15451361e-17],
        [ -2.77090007e-22,   2.33377864e-21,  -7.32050808e-01]]), 4)

In [10]:
shiftQR(B,1e-10, symmetric=True)

([-0.73205080756887753, -2.0000000000000009, 2.7320508075688776], 4)